In [60]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.preprocessing import MinMaxScaler

import json

In [108]:
n_clusters = 8
n_components = 2


# Load Dataset
df = pd.read_csv("BankChurners.csv")
df = df.iloc[:, :-2]
df.set_index('CLIENTNUM', inplace=True)

# One-hot Encoding
dfOrigin = df.copy()
df = pd.get_dummies(df)

# normalize
scaler = MinMaxScaler()
dfNorm = scaler.fit_transform(df)

# Kmeans Clustering
km = MiniBatchKMeans(n_clusters=n_clusters, random_state=2021)
dfPred = km.fit_predict(dfNorm)

df['grp'] = dfPred
dfOrigin['grp'] = dfPred

p = PCA(n_components=n_components)
dfPca = p.fit_transform(dfNorm)
dfPca = pd.DataFrame(dfPca, index=dfOrigin.index, columns=['x', 'y'])
dfPca['grp'] = dfOrigin.grp

t1 = pd.DataFrame(dfPca.groupby('grp').mean())
t2 = pd.DataFrame(dfPca.groupby('grp').size(), columns=['r'])
t2 = t2.apply(lambda x: x/np.array(t2).sum()*180, axis=0)
t3 = pd.merge(t1, t2, left_index=True, right_index=True)
clusterValue = t3.to_json(orient='records')


In [115]:
clusterValue

'[{"x":-0.9275793151,"y":-0.1222700627,"r":42.3560778118},{"x":0.7414916716,"y":-0.6119883027,"r":24.6351338007},{"x":-0.7542854566,"y":0.1044583803,"r":17.0810704058},{"x":-0.3625294273,"y":0.6969212428,"r":16.3878740002},{"x":0.9183993499,"y":0.7090956103,"r":20.2804384319},{"x":-0.4927698037,"y":-0.1776714155,"r":24.795102202},{"x":1.0063014424,"y":-0.4856346781,"r":21.2935716402},{"x":0.9120183491,"y":0.5630274431,"r":13.1707317073}]'

In [20]:
# data preprocess

df = pd.read_csv("BankChurners.csv")
df = df.iloc[:,:-2]

In [21]:
df.set_index('CLIENTNUM', inplace=True)

In [22]:
dfOrigin = df.copy()

In [23]:
df = pd.get_dummies(df)

In [24]:
scaler = MinMaxScaler()
dfNorm = scaler.fit_transform(df)

In [25]:
# Kmeans Clustering
km = MiniBatchKMeans(n_clusters=8)
dfPred = km.fit_predict(dfNorm)

In [26]:
df['grp'] = dfPred
dfOrigin['grp'] = dfPred

In [27]:
# reduce to 2 dim

from sklearn.decomposition import PCA

n_components = 2

p = PCA(n_components=n_components)
dfPca = p.fit_transform(dfNorm)
dfPca = pd.DataFrame(dfPca, index=dfOrigin.index, columns=['pca'+str(i) for i in range(n_components)])
dfPca['grp'] = dfOrigin.grp

In [28]:
t1 = pd.DataFrame(dfPca.groupby('grp').mean())
t2 = pd.DataFrame(dfPca.groupby('grp').size(), columns=['c'])
t2 = t2.apply(lambda x: x/np.array(t2).sum(), axis=0)

In [29]:
t3 = pd.merge(t1, t2, left_index=True, right_index=True)

In [30]:
dfOrigin.columns

Index(['Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio',
       'grp'],
      dtype='object')

In [39]:
t = dfOrigin[dfOrigin.grp==0] \
        [['Customer_Age', 'Months_on_book', 'Credit_Limit', 'Total_Trans_Amt', 'Avg_Utilization_Ratio', 'Total_Trans_Ct', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon']].mean()

In [52]:
t.to_dict()

{'Customer_Age': 46.636649214659684,
 'Months_on_book': 36.158115183246075,
 'Credit_Limit': 4506.669738219891,
 'Total_Trans_Amt': 2792.875392670157,
 'Avg_Utilization_Ratio': 0.20657172774869093,
 'Total_Trans_Ct': 43.98848167539267,
 'Months_Inactive_12_mon': 2.695287958115183,
 'Contacts_Count_12_mon': 2.943455497382199}

In [53]:
dictSegment = {}
for i in range(8):
    t = dfOrigin[dfOrigin.grp==i] \
        [['Customer_Age', 'Months_on_book', 'Credit_Limit', 'Total_Trans_Amt', 'Avg_Utilization_Ratio', 'Total_Trans_Ct', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon']].mean()
    t = t.to_dict()
    
    dictSegment[i+1] = t

In [61]:
json.dumps(dictSegment)

'{"1": {"Customer_Age": 46.636649214659684, "Months_on_book": 36.158115183246075, "Credit_Limit": 4506.669738219891, "Total_Trans_Amt": 2792.875392670157, "Avg_Utilization_Ratio": 0.20657172774869093, "Total_Trans_Ct": 43.98848167539267, "Months_Inactive_12_mon": 2.695287958115183, "Contacts_Count_12_mon": 2.943455497382199}, "2": {"Customer_Age": 46.09988385598142, "Months_on_book": 35.86062717770035, "Credit_Limit": 11552.567363530776, "Total_Trans_Amt": 4418.275261324042, "Avg_Utilization_Ratio": 0.22418815331010478, "Total_Trans_Ct": 61.47038327526133, "Months_Inactive_12_mon": 2.343786295005807, "Contacts_Count_12_mon": 2.5365853658536586}, "3": {"Customer_Age": 45.98193411264612, "Months_on_book": 35.5249734325186, "Credit_Limit": 7910.00903294367, "Total_Trans_Amt": 4834.489904357067, "Avg_Utilization_Ratio": 0.2938682252922426, "Total_Trans_Ct": 73.24548352816153, "Months_Inactive_12_mon": 2.2996811902231666, "Contacts_Count_12_mon": 2.343251859723698}, "4": {"Customer_Age": 45

In [86]:
dictGender = {}

for i in range(8):
        t = pd.DataFrame(dfOrigin[dfOrigin.grp == i].groupby(
            "Gender").size(), columns=['c'])
        t = t.apply(lambda x: x/np.array(t).sum(), axis=0)
        t.sort_values(by='c', ascending=False, inplace=True)
        gender_mf = t.index[0]
        if gender_mf == 'F':
            gender_mf = 'Female'
        else:
            gender_mf = 'Male'
        gender_ratio = '{:.1f}'.format(t.iloc[0, 0]*100)

        dictSegment[i+1] = t
        dictGender[i+1] = {'gender_mf': gender_mf,
                           'gender_ratio': gender_ratio}

In [84]:
dictGender = json.dumps(dictGender)

In [85]:
dictGender

'{"1": {"gender_mf": "Female", "gender_ratio": "97.4"}, "2": {"gender_mf": "Male", "gender_ratio": "100.0"}, "3": {"gender_mf": "Female", "gender_ratio": "100.0"}, "4": {"gender_mf": "Male", "gender_ratio": "100.0"}, "5": {"gender_mf": "Male", "gender_ratio": "100.0"}, "6": {"gender_mf": "Female", "gender_ratio": "100.0"}, "7": {"gender_mf": "Female", "gender_ratio": "100.0"}, "8": {"gender_mf": "Male", "gender_ratio": "100.0"}}'

In [94]:
dictGender[8]

{'gender_mf': 'Male', 'gender_ratio': '100.0'}

In [80]:
dictGender[1]

['F', '97.4']

In [ ]:
pd.DataFrame(dfOrigin[dfOrigin.grp == i].groupby(
            "Gender").size(), columns=['c'])

In [95]:
pd.DataFrame(dfOrigin[dfOrigin.grp == 1].groupby("Gender").size(), columns=['c'])

,c
Gender,
M,861


In [104]:
t3 = pd.DataFrame(dfOrigin.groupby("grp").size(), columns=['c'])
t3 = t3.apply(lambda x: x/np.array(t3).sum()*100, axis=0)
t3 = t3.to_dict()
t3 = t3['c']

In [105]:
t3

{0: 9.430236002764886,
 1: 8.502024291497975,
 2: 9.291991705342156,
 3: 13.755307593561767,
 4: 9.519107336822357,
 5: 20.134294460353512,
 6: 14.298410190579638,
 7: 15.068628419077712}

In [106]:
clusterValue

NameError: name 'clusterValue' is not defined

In [236]:
t3.to_json(orient='records')

'[{"pca0":-0.7760367656,"pca1":0.7471545716,"c":0.2098350943},{"pca0":-0.757035237,"pca1":-0.4844752519,"c":0.3192455811},{"pca0":0.8588027106,"pca1":-0.6563047136,"c":0.2207958922},{"pca0":0.8591749653,"pca1":0.5709049997,"c":0.2501234324}]'

In [145]:
# Gender
t = pd.DataFrame(dfOrigin.groupby("Gender").size(), columns=['c'])
t = t.apply(lambda x: x/np.array(t).sum(), axis=0)
t.sort_values(by='c', ascending=False, inplace=True)
gender_mf = t.index[0]
gender_ratio = '{:.1f}'.format(t.iloc[0, 0]*100)

In [164]:
# Age
dfOrigin['Customer_Age_g'] = dfOrigin.Customer_Age//10*10
t = pd.DataFrame(dfOrigin.groupby("Customer_Age_g").size(), columns=['c'])
t = t.apply(lambda x: x/np.array(t).sum(), axis=0)
t.sort_values(by='c', ascending=False, inplace=True)
age_g = t.index[0]
age_ratio_ratio = '{:.1f}'.format(t.iloc[0, 0]*100)

In [172]:
# MOB
mob = '{:.1f}'.format(dfOrigin.Months_on_book.mean())

In [174]:
# credit Limit
Credit_Limit = '{:.1f}'.format(dfOrigin.Credit_Limit.mean())

In [177]:
# Total_trans Amt
Usage = '{:.1f}'.format(dfOrigin.Total_Trans_Amt.mean())

In [180]:
# average util ratio
utilRatio = '{:.1f}'.format(dfOrigin.Avg_Utilization_Ratio.mean()*100)

In [186]:
# average open to buy
avgCnt = '{:.1f}'.format(dfOrigin.Total_Trans_Ct.mean())

In [188]:
# average open to buy
avgMthInactive = '{:.1f}'.format(dfOrigin.Months_Inactive_12_mon.mean())

In [189]:
# average open to buy
avgContact = '{:.1f}'.format(dfOrigin.Contacts_Count_12_mon.mean())

In [ ]:
# average open to buy
avgMthInactive = '{:.1f}'.format(dfOrigin.Months_Inactive_12_mon.mean())

In [191]:
dfOrigin.columns

Index(['Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'grp',
       'Customer_Age_g'],
      dtype='object')

In [193]:
t = pd.DataFrame(dfOrigin.groupby("Marital_Status").size(), columns=['c'])
t = t.apply(lambda x: x/np.array(t).sum(), axis=0)


In [202]:
list(t.c)

[0.07386195319443073,
 0.4628221585859583,
 0.3893551890984497,
 0.07396069912116125]

In [203]:
dfOrigin.columns

Index(['Attrition_Flag', 'Customer_Age', 'Gender', 'Dependent_count',
       'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category',
       'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon',
       'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal',
       'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt',
       'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'grp',
       'Customer_Age_g'],
      dtype='object')

In [198]:
r = t.to_json(orient='index')

'{"Divorced":{"c":0.0738619532},"Married":{"c":0.4628221586},"Single":{"c":0.3893551891},"Unknown":{"c":0.0739606991}}'

In [ ]:
>>> result = df.to_json(orient="split")
>>> parsed = json.loads(result)
>>> json.dumps(parsed, indent=4) 